In [2]:
from cassandra.cluster import Cluster
from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement
import os

2) Перевірити правильність конфігурації за допомогою
nodetool status

In [5]:
os.system('docker exec cassandra-1 nodetool status')

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load       Tokens  Owns (effective)  Host ID                               Rack 
UN  172.17.0.4  75.37 KiB  16      35.7%             7ffca09c-b06c-470d-bb7f-20c2383cd388  rack1
UN  172.17.0.3  75.37 KiB  16      31.6%             ac10746a-1d06-4484-adeb-6aca272b421d  rack1
UN  172.17.0.2  99.18 KiB  16      32.7%             2c97dff5-9efe-4670-9689-b69310409def  rack1



0

In [141]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

3) Викоритовуючи  cqlsh, створити три Keyspace з replication factor 1, 2, 3

In [142]:
session.execute("""
CREATE KEYSPACE repl_1
WITH replication = {
    'class': 'SimpleStrategy',
    'replication_factor': 1
    };""").one()
session.execute("""
CREATE KEYSPACE repl_2
WITH replication = {
    'class': 'SimpleStrategy',
    'replication_factor': 2
    };""").one()
session.execute("""
CREATE KEYSPACE repl_3
WITH replication = {
    'class': 'SimpleStrategy',
    'replication_factor': 3
    };""").one()

4) В кожному з кейспейсів створити таблиці

In [143]:
session.execute("USE repl_1").one()
session.execute("""
CREATE TABLE IF NOT EXISTS item1 (
    category text,
    price int,
    model text,
    PRIMARY KEY (category, price)
)
WITH CLUSTERING ORDER BY (price ASC);""").one()

In [144]:
session.execute("USE repl_2").one()
session.execute("""
CREATE TABLE IF NOT EXISTS item2 (
    category text,
    price int,
    producer text,
    PRIMARY KEY (category, price)
)
WITH CLUSTERING ORDER BY (price ASC);""").one()

In [145]:
session.execute("USE repl_3").one()
session.execute("""
CREATE TABLE IF NOT EXISTS item3 (
    category text,
    price int,
    material text,
    PRIMARY KEY (category, price)
)
WITH CLUSTERING ORDER BY (price ASC);""").one()

5) Спробуйте писати і читати на / та з різних нод.

Створюємо конекти до інших нод кластеру

In [100]:
cluster2 = Cluster(['localhost'], port=9043)
session2 = cluster2.connect()

In [104]:
cluster3 = Cluster(['localhost'], port=9044)
session3 = cluster3.connect()

Запис і вичитування даних з першої ноди

In [13]:
session.execute("USE repl_1").one()
session.execute("""
INSERT INTO item1 (
    category, price, model
) VALUES (
    'Phone',
    570,
    'iPhone 14'
    );""")

In [15]:
result = session.execute("""
SELECT *
FROM item1
WHERE category = 'Phone'
ORDER BY price;""")
if result:
    for row in result:
        print(*row, '\n')

Phone 570 iPhone 14 



Запис і вичитування даних з другої ноди

In [16]:
session2.execute("USE repl_1").one()
session2.execute("""
INSERT INTO item1 (
    category, price, model
) VALUES (
    'Phone',
    630,
    'iPhone 12'
    );""")

In [17]:
result = session2.execute("""
SELECT *
FROM item1
WHERE category = 'Phone'
ORDER BY price;""")
if result:
    for row in result:
        print(*row, '\n')

Phone 570 iPhone 14 

Phone 630 iPhone 12 



Запис і вичитування даних з третої ноди

In [18]:
session3.execute("USE repl_1").one()
session3.execute("""
INSERT INTO item1 (
    category, price, model
) VALUES (
    'Phone',
    470,
    'iPhone 13'
    );""")

In [19]:
result = session3.execute("""
SELECT *
FROM item1
WHERE category = 'Phone'
ORDER BY price;""")
if result:
    for row in result:
        print(*row, '\n')

Phone 470 iPhone 13 

Phone 570 iPhone 14 

Phone 630 iPhone 12 



6) Вставте дані в створені таблиці і подивіться на їх розподіл по вузлах кластера (для кожного з кейспесов - nodetool status)

In [20]:
session.execute("USE repl_1").one()
session.execute("""
INSERT INTO item1 (
    category, price, model
) VALUES (
    'Watch',
    240,
    'Series SE'
    );""")
session.execute("""
INSERT INTO item1 (
    category, price, model
) VALUES (
    'Watch',
    400,
    'Series 8'
    );""")

In [22]:
os.system('docker exec cassandra-1 nodetool status repl_1')

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.17.0.4  144.26 KiB  16      35.7%             7ffca09c-b06c-470d-bb7f-20c2383cd388  rack1
UN  172.17.0.3  144.31 KiB  16      31.6%             ac10746a-1d06-4484-adeb-6aca272b421d  rack1
UN  172.17.0.2  169.4 KiB   16      32.7%             2c97dff5-9efe-4670-9689-b69310409def  rack1



0

In [21]:
session.execute("USE repl_2").one()
session.execute("""
INSERT INTO item2 (
    category, price, producer
) VALUES (
    'Phone',
    1039,
    'Samsung'
    );""")
session.execute("""
INSERT INTO item2 (
    category, price, producer
) VALUES (
    'Phone',
    940,
    'Samsung'
    );""")

In [23]:
os.system('docker exec cassandra-1 nodetool status repl_2')

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.17.0.4  144.26 KiB  16      76.0%             7ffca09c-b06c-470d-bb7f-20c2383cd388  rack1
UN  172.17.0.3  144.31 KiB  16      59.3%             ac10746a-1d06-4484-adeb-6aca272b421d  rack1
UN  172.17.0.2  169.4 KiB   16      64.7%             2c97dff5-9efe-4670-9689-b69310409def  rack1



0

In [24]:
session.execute("USE repl_3").one()
session.execute("""
INSERT INTO item3 (
    category, price, material
) VALUES (
    'Headphones',
    120,
    'Plastic'
    );""")
session.execute("""
INSERT INTO item3 (
    category, price, material
) VALUES (
    'Headphones',
    318,
    'Aluminium'
    );""")

In [25]:
os.system('docker exec cassandra-1 nodetool status repl_3')

Datacenter: datacenter1
Status=Up/Down
|/ State=Normal/Leaving/Joining/Moving
--  Address     Load        Tokens  Owns (effective)  Host ID                               Rack 
UN  172.17.0.4  144.26 KiB  16      100.0%            7ffca09c-b06c-470d-bb7f-20c2383cd388  rack1
UN  172.17.0.3  144.31 KiB  16      100.0%            ac10746a-1d06-4484-adeb-6aca272b421d  rack1
UN  172.17.0.2  169.4 KiB   16      100.0%            2c97dff5-9efe-4670-9689-b69310409def  rack1



0

7) Для якогось запису з кожного з кейспейсу виведіть ноди на яких зберігаються дані

In [28]:
os.system('docker exec cassandra-1 nodetool getendpoints repl_1 item1 Watch')

172.17.0.2


0

In [29]:
os.system('docker exec cassandra-1 nodetool getendpoints repl_2 item2 Phone')

172.17.0.3
172.17.0.2


0

In [31]:
os.system('docker exec cassandra-1 nodetool getendpoints repl_3 item3 Headphones')

172.17.0.3
172.17.0.4
172.17.0.2


0

8) Відключиіть одну з нод. Для кожного з кейспейсів визначить з якими рівнями consistency можемо читати та писати, і які з них забезпечують strong consistency

In [94]:
for keyspace in ("1", "2", "3"):
    for cons in (ConsistencyLevel.LOCAL_SERIAL, ConsistencyLevel.SERIAL, ConsistencyLevel.LOCAL_ONE, ConsistencyLevel.THREE,
                 ConsistencyLevel.TWO, ConsistencyLevel.ONE, ConsistencyLevel.LOCAL_QUORUM, ConsistencyLevel.QUORUM,
                 ConsistencyLevel.EACH_QUORUM, ConsistencyLevel.ALL):
        session.execute(f"USE repl_{keyspace}")
        query = SimpleStatement(
            f"""
            SELECT *
            FROM item{keyspace}
            WHERE category = 'Phone'
            ORDER BY price;""",
            consistency_level=cons)
        try:
            session.execute(query)
        except Exception as e:
            print(f"For keyspace repl_{keyspace} on READ consistency is unavailable: {str(e).split('consistency')[1].split(' ')[2][:-1]}\n")


For keyspace repl_1 on READ consistency is unavailable: THREE

For keyspace repl_1 on READ consistency is unavailable: TWO

For keyspace repl_2 on READ consistency is unavailable: THREE

For keyspace repl_3 on READ consistency is unavailable: THREE

For keyspace repl_3 on READ consistency is unavailable: ALL



In [95]:
for keyspace in ("1", "2", "3"):
    for cons in (ConsistencyLevel.ANY, ConsistencyLevel.LOCAL_ONE, ConsistencyLevel.THREE, ConsistencyLevel.TWO,
                 ConsistencyLevel.ONE, ConsistencyLevel.LOCAL_QUORUM, ConsistencyLevel.QUORUM, ConsistencyLevel.EACH_QUORUM,
                 ConsistencyLevel.ALL):
        session.execute(f"USE repl_{keyspace}")
        query = SimpleStatement(
            f"""
            INSERT INTO item{keyspace} (
                category, price
            ) VALUES (
                'Category{keyspace}',
                {keyspace}351
                );""",
            consistency_level=cons)
        try:
            session.execute(query)
        except Exception as e:
            print(f"For keyspace repl_{keyspace} on WRITE consistency is unavailable: {str(e).split('consistency')[1].split(' ')[2][:-1]}\n")

For keyspace repl_1 on WRITE consistency is unavailable: LOCAL_ONE

For keyspace repl_1 on WRITE consistency is unavailable: THREE

For keyspace repl_1 on WRITE consistency is unavailable: TWO

For keyspace repl_1 on WRITE consistency is unavailable: ONE

For keyspace repl_1 on WRITE consistency is unavailable: LOCAL_QUORUM

For keyspace repl_1 on WRITE consistency is unavailable: QUORUM

For keyspace repl_1 on WRITE consistency is unavailable: EACH_QUORUM

For keyspace repl_1 on WRITE consistency is unavailable: ALL

For keyspace repl_2 on WRITE consistency is unavailable: THREE

For keyspace repl_2 on WRITE consistency is unavailable: TWO

For keyspace repl_2 on WRITE consistency is unavailable: LOCAL_QUORUM

For keyspace repl_2 on WRITE consistency is unavailable: QUORUM

For keyspace repl_2 on WRITE consistency is unavailable: EACH_QUORUM

For keyspace repl_2 on WRITE consistency is unavailable: ALL

For keyspace repl_3 on WRITE consistency is unavailable: THREE

For keyspace repl_

9) Зробить так щоб три ноди працювали, але не бачили одна одну по мережі (відключити зв'язок між ними)

docker network disconnect bridge cassandra-1
docker network disconnect bridge cassandra-2
docker network disconnect bridge cassandra-3

10) Для кейспейсу з replication factor 3 задайте рівень consistency рівним 1. Виконайте запис одного й того самого значення, з однаковим primary key, але різними іншими значенням на кожну з нод (тобто створіть конфлікт)


In [117]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

session.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    INSERT INTO item3 (
        category, price, material
    ) VALUES (
        'ConflictCategory2',
        123,
        'material1'
        );""",
    consistency_level=ConsistencyLevel.ONE)
session.execute(query)

In [119]:
cluster2 = Cluster(['localhost'], port=9043)
session2 = cluster2.connect()

session2.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    INSERT INTO item3 (
        category, price, material
    ) VALUES (
        'ConflictCategory2',
        123,
        'material2'
        );""",
    consistency_level=ConsistencyLevel.ONE)
session2.execute(query)

In [121]:
cluster3 = Cluster(['localhost'], port=9044)
session3 = cluster3.connect()

session3.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    INSERT INTO item3 (
        category, price, material
    ) VALUES (
        'ConflictCategory2',
        123,
        'material3'
        );""",
    consistency_level=ConsistencyLevel.ONE)
session3.execute(query)

11) Об’єднайте ноди в кластер і визначте яке значення було прийнято кластером та за яким принципом

In [140]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

session.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    SELECT *
    FROM item3
    WHERE category = 'ConflictCategory2'
    ORDER BY price;""",
    consistency_level=ConsistencyLevel.QUORUM)
result = session.execute(query)
if result:
    for row in result:
        print(*row, '\n')

ConflictCategory2 123 material3 



Було обрано значення записане на останню ноду, бо касандра приймає значення зі найбільшим timestamp

12) Перевірте поведінку lightweight transactions для попередніх пунктів у розділеному та не розділеному на три частини кластері

В розділеному стані

In [3]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

session.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    INSERT INTO item3 (
        category, price, material
    ) VALUES (
        'ConflictCategory2',
        123,
        'material1'
        )
    IF NOT EXISTS;""",
    consistency_level=ConsistencyLevel.ONE)
session.execute(query)

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level QUORUM" info={\'consistency\': \'QUORUM\', \'required_replicas\': 2, \'alive_replicas\': 1}')})

In [4]:
cluster2 = Cluster(['localhost'], port=9043)
session2 = cluster2.connect()

session2.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    INSERT INTO item3 (
        category, price, material
    ) VALUES (
        'ConflictCategory2',
        123,
        'material2'
        )
    IF NOT EXISTS;""",
    consistency_level=ConsistencyLevel.ONE)
session2.execute(query)

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level QUORUM" info={\'consistency\': \'QUORUM\', \'required_replicas\': 2, \'alive_replicas\': 1}')})

In [5]:
cluster3 = Cluster(['localhost'], port=9044)
session3 = cluster3.connect()

session3.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    INSERT INTO item3 (
        category, price, material
    ) VALUES (
        'ConflictCategory2',
        123,
        'material3'
        )
    IF NOT EXISTS;""",
    consistency_level=ConsistencyLevel.ONE)
session3.execute(query)

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level QUORUM" info={\'consistency\': \'QUORUM\', \'required_replicas\': 2, \'alive_replicas\': 1}')})

In [6]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

session.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    SELECT *
    FROM item3
    WHERE category = 'ConflictCategory2'
    ORDER BY price;""",
    consistency_level=ConsistencyLevel.QUORUM)
result = session.execute(query)
if result:
    for row in result:
        print(*row, '\n')

NoHostAvailable: ('Unable to complete the operation against any hosts', {<Host: 127.0.0.1:9042 datacenter1>: Unavailable('Error from server: code=1000 [Unavailable exception] message="Cannot achieve consistency level QUORUM" info={\'consistency\': \'QUORUM\', \'required_replicas\': 2, \'alive_replicas\': 1}')})

В не розділеному стані

In [157]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

session.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    INSERT INTO item3 (
        category, price, material
    ) VALUES (
        'ConflictCategory3',
        123,
        'material1'
        )
    IF NOT EXISTS;""")
session.execute(query)

In [159]:
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

session.execute(f"USE repl_3")
query = SimpleStatement(
    f"""
    SELECT *
    FROM item3
    WHERE category = 'ConflictCategory3'
    ORDER BY price;""",
    consistency_level=ConsistencyLevel.QUORUM)
result = session.execute(query)
if result:
    for row in result:
        print(*row, '\n')

ConflictCategory3 123 material1 

